In [13]:
library(ggplot2)
library(tidyverse)
library(lubridate)
library(Seurat)

library(dplyr)
library(AnnotationDbi)

In [14]:
library(gridExtra)

In [15]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

In [16]:
filename <- "/gpfs/home/meyin/published_data/parabiosis/pb_combined.rds?download=1"
origdata <- readRDS(filename)

In [17]:
origdata <- subset(origdata, subset = Celltype != "Doublet")

In [18]:
sem <- function(data) {
    numpoints <- length(data[[4]])

    data <- data[!is.infinite(data$log_value),]

    std_dev <- sd(data$log_value, na.rm=TRUE)
    
    standard_error <- std_dev / sqrt(numpoints)

    return(standard_error)
}

In [19]:
calcmean <- function(data) {
    data <- data[!is.infinite(data$log_value),]

    mean <- mean(data$log_value, na.rm=TRUE)
    
    return(mean)
}

In [20]:
# Takes in a sparse matrix row for a specific gene from the aged group
#    and a pre-calculated average for the young reference group
calculateTransDrift <- function(geneAged, geneYoungRefAvg){
    geneAgedAvg <- mean(geneAged)
    
    foldCh <- geneAgedAvg/geneYoungRefAvg  

}

In [21]:
# Given a list of gene names in a gene set and a matrix, remove all rows that
#    are associated with a gene that isn't in the gene set names list.
filter_matrix <- function(matrix, gene_set_names) {
  gene_names <- rownames(matrix)
  
  rows_to_keep <- gene_names %in% gene_set_names
  
  filtered_matrix <- matrix[rows_to_keep, ]
  
  return(filtered_matrix)
}

In [22]:
# Cell type specific trans. drift. var.

In [23]:
celltypes <- unique(origdata$Celltype)

In [27]:
idToGroup <- c(
    'BC2' = "Y_Het", 
    'BC3' = "O_Iso", 
    'BC4' = "O_Iso", 
    'BC6' = "O_Iso", 
    'BC1' = "O_Het", 
    'BC5' = "O_Iso",
    'BC7' = "O_Het", 
    'BC13' = "O_Het",
    'BC11' = "Y_Iso", 
    'BC9' = "O_Iso",
    'BC12' = "Y_Iso", 
    'BC8' = "Y_Het",
    'BC14' = "Y_Het", 
    'BC10' = "O_Iso",
    'BC23' = "Y_Iso",
    'BC22' = "Y_Het",
    'BC21' = "O_Het", 
    'BC24' = "Y_Iso");

In [100]:
idToNum <- c(
    'BC2' = 3, 
    'BC3' = 0, 
    'BC4' = 0, 
    'BC6' = 0, 
    'BC1' = 1, 
    'BC5' = 0,
    'BC7' = 1, 
    'BC13' = 1,
    'BC11' = 2, 
    'BC9' = 0,
    'BC12' = 2, 
    'BC8' = 3,
    'BC14' = 3, 
    'BC10' = 0,
    'BC23' = 2,
    'BC22' = 3,
    'BC21' = 1, 
    'BC24' = 2);

In [31]:
idToColor <- c(
    'BC2' = "#556F44", 
    'BC3' = "#78CDD7", 
    'BC4' = "#78CDD7", 
    'BC6' = "#78CDD7", 
    'BC1' = "#235789", 
    'BC5' = "#78CDD7",
    'BC7' = "#235789", 
    'BC13' = "#235789",
    'BC11' = "#A7A650", 
    'BC9' = "#78CDD7",
    'BC12' = "#A7A650", 
    'BC8' = "#556F44",
    'BC14' = "#556F44", 
    'BC10' = "#78CDD7",
    'BC23' = "#A7A650" ,
    'BC22' = "#556F44",
    'BC21' = "#235789", 
    'BC24' = "#A7A650");

In [32]:
filter_matrix <- function(matrix, gene_set_names) {
  filtered <- matrix[gene_set_names, , drop = FALSE]
  return(filtered)
}

In [53]:
data@meta.data <- data@meta.data %>%
  mutate(AgeCond = paste0(substr(Age, 1, 1), "_", Type))

In [61]:
data@meta.data <- data@meta.data %>%
  mutate(ID = sub("-.*", "", hash.ID))

In [107]:
for (celltype in celltypes) {
    figures_logpl <- list()
    figures_pl <- list()
    figures_logdot <- list()
    figures_dot <- list()

    counter <- 1
    
    data <- subset(origdata, subset = Celltype == celltype)

    data@meta.data <- data@meta.data %>%
      mutate(AgeCond = paste0(substr(Age, 1, 1), "_", Type))

    data@meta.data <- data@meta.data %>%
      mutate(ID = sub("-.*", "", hash.ID))

    file_list <- list.files("/gpfs/home/meyin/gene_sets/cell_components/gene_sets_common_names", pattern = "\\.txt$", full.names = TRUE)

    for (gene_set_name_pathway in file_list)
    {
        prefix <- strsplit(strsplit(gene_set_name_pathway, "/")[[1]][length(strsplit(gene_set_name_pathway, "/")[[1]])][[1]], "_gene_names")[[1]][1]
        prefix <- paste(celltype, prefix)

        # Split off young control data
        ycontroldata <- subset(data, subset = AgeCond == "Y_Iso")
        
        ycontroldata <- SplitObject(ycontroldata, split.by = "ID")
        
        # Create dataset w no young control data
        dataNoYControl <- subset(data, subset = AgeCond != "Y_Iso")

        dataNoYControl <- subset(dataNoYControl, subset = ID != "NA")
        
        availIDs <- na.omit(unique(dataNoYControl$ID))
        
        dataNoYControl <- SplitObject(dataNoYControl, split.by = "ID")
        
        # Convert list of Seurat objects into sparse matrices
        dataNoYControl_initial_matrices <- lapply(dataNoYControl, function(x) GetAssayData(x, slot = "data"))
        ycontroldata_initial_matrices <- lapply(ycontroldata, function(x) GetAssayData(x, slot = "data"))
    
        # Take average of each gene for each matrix in ycontroldata_initial_matrices
        ycontrolaverages <- lapply(ycontroldata_initial_matrices, function(matrix) {
        apply(matrix, MARGIN = 1, FUN = function(x) mean(x))
        })
        
        # Turn results into dataframe; each row is one organism
        ycontrolaverages <- do.call(rbind, lapply(ycontrolaverages, unlist))
        ycontrolaverages <- as.data.frame(ycontrolaverages)
    
        # Get gene set names
        gene_set_names <- read.table(gene_set_name_pathway, sep = " ", header = FALSE)[, 2]
        gene_set_names <- gene_set_names[gene_set_names %in% colnames(ycontrolaverages)]

        # Keep the columns indicated by the gene_set_names
        ycontrolaverages <- ycontrolaverages %>% dplyr::select(all_of(gene_set_names))

        # Find the average for each column
        ycontrolaverages <- colMeans(ycontrolaverages)
        
        # Turn into list
        ycontrolaverages <- as.list(ycontrolaverages)

        # Remove irrelevant genes from dataNoYControl_initial_matrices
        dataNoYControl_initial_matrices <- lapply(dataNoYControl_initial_matrices, filter_matrix, gene_set_names)

        # calculate transcriptional drift
        result <- c()
        for (matrix_idx in seq_along(dataNoYControl_initial_matrices)) {
          matrix <- dataNoYControl_initial_matrices[[matrix_idx]]
          
          matrix_result <- vector("list", nrow(matrix))
          
          for (row_idx in seq_len(nrow(matrix))) {
            row <- matrix[row_idx, ]
            
            matrix_result[[row_idx]] <- calculateTransDrift(row, ycontrolaverages[[row_idx]])
          }
          
          result[[matrix_idx]] <- matrix_result
        }
            
        
        result_df <- data.frame(
          Matrix = rep(seq_along(result), sapply(result, length)),
          Value = unlist(result)
        )
        
        result_df$log_value <- log(result_df$Value)
        
        increment <- dim(dataNoYControl_initial_matrices[[1]])[1]
            
        left <- 1
        right <- increment

        colorbygroup <- c();
            
        for (i in 1:length(availIDs))
        {
            result_df$ID[left:right] <- availIDs[i]
            left <- right + 1
            right <- right + increment

            colorbygroup = c(colorbygroup, idToColor[availIDs[i]])
        }

        result_df_noNA <- na.omit(result_df)
        result_df_noNA <- result_df_noNA[!is.infinite(result_df_noNA$log_value),]
        
        # Calculate Drift Variance
        # try cross validation later
        
        driftvariances <- result_df_noNA %>%
          group_by(ID) %>%
          summarise(log_drift_variance = var(log_value), drift_variance = var(Value))
        
        result_df_noNA <- result_df_noNA %>%
          left_join(driftvariances, by = "ID")   
        
        idlist <- unique(result_df_noNA$ID);
        
        experimentalgroup <- c();
        experimentalgroupnum <- c();
        for (id in idlist)
        {
            experimentalgroup = c(experimentalgroup, idToGroup[id])
            experimentalgroupnum = c(experimentalgroupnum, idToNum[id])
        }
    
        driftvariances <- driftvariances %>%
            mutate(ID=idlist, experimentalGroup = experimentalgroup, experimentalgroupnum = experimentalgroupnum)
    
        driftvariances <- data.frame(driftvariances)
        
        logpl <- ggplot(driftvariances, aes(x=reorder(ID, experimentalgroupnum), y=log_drift_variance, fill = ID)) + 
            geom_col(aes(fill = ID)) +
            ggtitle(paste("Log Transcriptional Drift Variance in", prefix)) +
            labs(y = "Drift Variance", x = "Group") +
            scale_fill_manual(values = colorbygroup)
            
        #ggsave(paste(prefix, "LogDriftVariance.png"), logpl, width = 10, height = 10)
            
        figures_logpl[[counter]] <- logpl
        
        pl <- ggplot(driftvariances, aes(x=reorder(ID, experimentalgroupnum), y=drift_variance, fill = ID)) + 
            geom_col(aes(fill = ID)) +
            ggtitle(paste("Transcriptional Drift Variance in", prefix)) +
            labs(y = "Drift Variance", x = "Group") +
            scale_fill_manual(values = colorbygroup)
            
        #ggsave(paste(prefix, "DriftVariance.png"), pl, width = 10, height = 10)

        figures_pl[[counter]] <- pl
        
        logdot <- ggplot(driftvariances) +
            geom_point(aes(x = experimentalGroup, y = log_drift_variance, color = experimentalGroup), size=5) +
            labs(title = paste("Log Transcriptional Drift Compared Across Age/Parabiosis Groups \n in", prefix))
        
        #ggsave(paste(prefix, "LogDriftVarianceDot.png"), logdot, width = 10, height = 10)

        figures_logdot[[counter]] <- logdot
        
        dot <- ggplot(driftvariances) +
            geom_point(aes(x = experimentalGroup, y = drift_variance, color = experimentalGroup), size=5) +
            labs(title = paste("Transcriptional Drift Compared Across Age/Parabiosis Groups \n in", prefix))
    
        #ggsave(paste(prefix, "DriftVarianceDot.png"), dot, width = 10, height = 10)

        figures_dot[[counter]] <- dot
        
        counter <- counter + 1
    }

    ggsave(paste0("driftbygenesets_cellularcomponents/", paste(celltype, "LogDriftVarianceAllGeneSets.png")), marrangeGrob(figures_logpl, nrow = 2, ncol = 8), width = 50, height = 30, limitsize = FALSE)
    ggsave(paste0("driftbygenesets_cellularcomponents/", paste(celltype, "DriftVarianceAllGeneSets.png")), marrangeGrob(figures_pl, nrow = 2, ncol = 8), width = 50, height = 30, limitsize = FALSE)
    ggsave(paste0("driftbygenesets_cellularcomponents/", paste(celltype, "LogDotDriftVarianceAllGeneSets.png")), marrangeGrob(figures_logdot, nrow = 2, ncol = 8), width = 50, height = 30, limitsize = FALSE)
    ggsave(paste0("driftbygenesets_cellularcomponents/", paste(celltype, "DotDriftVarianceAllGeneSets.png")), marrangeGrob(figures_dot, nrow = 2, ncol = 8), width = 50, height = 30, limitsize = FALSE)

}

In [116]:
for (celltype in celltypes) {
    figures_logpl <- list()
    figures_pl <- list()
    figures_logdot <- list()
    figures_dot <- list()

    counter <- 1
    
    data <- subset(origdata, subset = Celltype == celltype)

    data@meta.data <- data@meta.data %>%
      mutate(AgeCond = paste0(substr(Age, 1, 1), "_", Type))

    data@meta.data <- data@meta.data %>%
      mutate(ID = sub("-.*", "", hash.ID))

    file_list <- list.files("/gpfs/home/meyin/gene_sets/biological_processes/gene_sets_common_names", pattern = "\\.txt$", full.names = TRUE)

    for (gene_set_name_pathway in file_list)
    {
        prefix <- strsplit(strsplit(gene_set_name_pathway, "/")[[1]][length(strsplit(gene_set_name_pathway, "/")[[1]])][[1]], "_gene_names")[[1]][1]
        prefix <- paste(celltype, prefix)

        # Split off young control data
        ycontroldata <- subset(data, subset = AgeCond == "Y_Iso")
        
        ycontroldata <- SplitObject(ycontroldata, split.by = "ID")
        
        # Create dataset w no young control data
        dataNoYControl <- subset(data, subset = AgeCond != "Y_Iso")

        dataNoYControl <- subset(dataNoYControl, subset = ID != "NA")
        
        availIDs <- unique(dataNoYControl$ID)
        
        dataNoYControl <- SplitObject(dataNoYControl, split.by = "ID")
        
        # Convert list of Seurat objects into sparse matrices
        dataNoYControl_initial_matrices <- lapply(dataNoYControl, function(x) GetAssayData(x, slot = "data"))
        ycontroldata_initial_matrices <- lapply(ycontroldata, function(x) GetAssayData(x, slot = "data"))
    
        # Take average of each gene for each matrix in ycontroldata_initial_matrices
        ycontrolaverages <- lapply(ycontroldata_initial_matrices, function(matrix) {
        apply(matrix, MARGIN = 1, FUN = function(x) mean(x))
        })
        
        # Turn results into dataframe; each row is one organism
        ycontrolaverages <- do.call(rbind, lapply(ycontrolaverages, unlist))
        ycontrolaverages <- as.data.frame(ycontrolaverages)
    
        # Get gene set names
        gene_set_names <- read.table(gene_set_name_pathway, sep = " ", header = FALSE)[, 2]
        gene_set_names <- gene_set_names[gene_set_names %in% colnames(ycontrolaverages)]

        # Keep the columns indicated by the gene_set_names
        ycontrolaverages <- ycontrolaverages %>% dplyr::select(all_of(gene_set_names))

        # Find the average for each column
        ycontrolaverages <- colMeans(ycontrolaverages)
        
        # Turn into list
        ycontrolaverages <- as.list(ycontrolaverages)

        # Remove irrelevant genes from dataNoYControl_initial_matrices
        dataNoYControl_initial_matrices <- lapply(dataNoYControl_initial_matrices, filter_matrix, gene_set_names)

        # calculate transcriptional drift
        result <- c()
        for (matrix_idx in seq_along(dataNoYControl_initial_matrices)) {
          matrix <- dataNoYControl_initial_matrices[[matrix_idx]]
          
          matrix_result <- vector("list", nrow(matrix))
          
          for (row_idx in seq_len(nrow(matrix))) {
            row <- matrix[row_idx, ]
            
            matrix_result[[row_idx]] <- calculateTransDrift(row, ycontrolaverages[[row_idx]])
          }
          
          result[[matrix_idx]] <- matrix_result
        }
        
        result_df <- data.frame(
          Matrix = rep(seq_along(result), sapply(result, length)),
          Value = unlist(result)
        )
        
        result_df$log_value <- log(result_df$Value)
        
        increment <- dim(dataNoYControl_initial_matrices[[1]])[1]
            
        left <- 1
        right <- increment

        colorbygroup <- c();
            
        for (i in 1:length(availIDs))
        {
            result_df$ID[left:right] <- availIDs[i]
            left <- right + 1
            right <- right + increment

            colorbygroup = c(colorbygroup, idToColor[availIDs[i]])
        }

        result_df_noNA <- na.omit(result_df)
        result_df_noNA <- result_df_noNA[!is.infinite(result_df_noNA$log_value),]
        
        # Calculate Drift Variance
        # try cross validation later
        
        driftvariances <- result_df_noNA %>%
          group_by(ID) %>%
          summarise(log_drift_variance = var(log_value), drift_variance = var(Value))
        
        result_df_noNA <- result_df_noNA %>%
          left_join(driftvariances, by = "ID")   
        
        idlist <- unique(result_df_noNA$ID);
        
        experimentalgroup <- c();
        experimentalgroupnum <- c();
        for (id in idlist)
        {
            experimentalgroup = c(experimentalgroup, idToGroup[id])
            experimentalgroupnum = c(experimentalgroupnum, idToNum[id])
        }
    
        driftvariances <- driftvariances %>%
            mutate(ID=idlist, experimentalGroup = experimentalgroup, experimentalgroupnum = experimentalgroupnum)
    
        driftvariances <- data.frame(driftvariances)
        
        logpl <- ggplot(driftvariances, aes(x=reorder(ID, experimentalgroupnum), y=log_drift_variance, fill = ID)) + 
            geom_col(aes(fill = ID)) +
            ggtitle(paste("Log Transcriptional Drift Variance in \n", prefix)) +
            labs(y = "Drift Variance", x = "Group") +
            scale_fill_manual(values = colorbygroup)
            
        #ggsave(paste(prefix, "LogDriftVariance.png"), logpl, width = 10, height = 10)
            
        figures_logpl[[counter]] <- logpl
        
        pl <- ggplot(driftvariances, aes(x=reorder(ID, experimentalgroupnum), y=drift_variance, fill = ID)) + 
            geom_col(aes(fill = ID)) +
            ggtitle(paste("Transcriptional Drift Variance in \n", prefix)) +
            labs(y = "Drift Variance", x = "Group") +
            scale_fill_manual(values = colorbygroup)
            
        #ggsave(paste(prefix, "DriftVariance.png"), pl, width = 10, height = 10)

        figures_pl[[counter]] <- pl
        
        logdot <- ggplot(driftvariances) +
            geom_point(aes(x = experimentalGroup, y = log_drift_variance, color = experimentalGroup), size=5) +
            labs(title = paste("Log Transcriptional Drift Compared Across Age/Parabiosis Groups \n in", prefix))
        
        #ggsave(paste(prefix, "LogDriftVarianceDot.png"), logdot, width = 10, height = 10)

        figures_logdot[[counter]] <- logdot
        
        dot <- ggplot(driftvariances) +
            geom_point(aes(x = experimentalGroup, y = drift_variance, color = experimentalGroup), size=5) +
            labs(title = paste("Transcriptional Drift Compared Across Age/Parabiosis Groups \n in", prefix))
    
        #ggsave(paste(prefix, "DriftVarianceDot.png"), dot, width = 10, height = 10)

        figures_dot[[counter]] <- dot
        
        counter <- counter + 1
    }

    ggsave(paste0("driftbygenesets_biologicalprocesses/", paste(celltype, "LogDriftVarianceAllGeneSets.png")), marrangeGrob(figures_logpl, nrow = 3, ncol = 14), width = 70, height = 20, limitsize = FALSE)
    ggsave(paste0("driftbygenesets_biologicalprocesses/", paste(celltype, "DriftVarianceAllGeneSets.png")), marrangeGrob(figures_pl, nrow = 3, ncol = 14), width = 70, height = 20, limitsize = FALSE)
    ggsave(paste0("driftbygenesets_biologicalprocesses/", paste(celltype, "LogDotDriftVarianceAllGeneSets.png")), marrangeGrob(figures_logdot, nrow = 3, ncol = 14), width = 70, height = 20, limitsize = FALSE)
    ggsave(paste0("driftbygenesets_biologicalprocesses/", paste(celltype, "DotDriftVarianceAllGeneSets.png")), marrangeGrob(figures_dot, nrow = 3, ncol = 14), width = 70, height = 20, limitsize = FALSE)
}

Warning message:
“Removed 1 rows containing missing values (`position_stack()`).”
Warning message:
“Removed 1 rows containing missing values (`position_stack()`).”
Warning message:
“Removed 1 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 1 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 4 rows containing missing values (`position_stack()`).”
Warning message:
“Removed 4 rows containing missing values (`position_stack()`).”
Warning message:
“Removed 4 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 4 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 2 rows containing missing values (`position_stack()`).”
Warning message:
“Removed 1 rows containing missing values (`position_stack()`).”
Warning message:
“Removed 2 rows containing missing values (`position_stack()`).”
Warning message:
“Removed 1 rows containing missing values (`position_stack()`).”
Warning message:
“Removed 2 rows